In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import joblib
import json
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import sys
import os
import importlib

warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

# Add parent directory to path for imports
sys.path.append('..')

print("📦 Libraries imported successfully!")

In [ ]:
def get_latest_trained_models():
    """Find and return paths to the latest trained models using versioning."""
    model_dir = Path('.')
    
    # Find all training directories
    training_dirs = [d for d in model_dir.iterdir() if d.is_dir() and d.name.startswith('training_')]
    
    if not training_dirs:
        # Fallback to standard model locations
        print("⚠️  No training directories found, using standard model locations...")
        return {
            'winner_model_path': 'ufc_random_forest_model_tuned.joblib',
            'method_model_path': 'ufc_multiclass_model.joblib',
            'winner_cols_path': 'winner_model_columns.json',
            'method_cols_path': 'method_model_columns.json',
            'fighters_data_path': 'ufc_fighters_engineered_corrected.csv',
            'version': 'standard'
        }
    
    # Sort by directory name to get the latest (assuming timestamp format)
    latest_training_dir = sorted(training_dirs, key=lambda x: x.name)[-1]
    version = latest_training_dir.name.replace('training_', '')
    
    print(f"✅ Using latest training version: {version}")
    
    # Build paths for versioned models
    return {
        'winner_model_path': latest_training_dir / f'ufc_winner_model_tuned_{version}.joblib',
        'method_model_path': latest_training_dir / f'ufc_method_model_{version}.joblib',
        'winner_cols_path': latest_training_dir / f'winner_model_columns_{version}.json',
        'method_cols_path': latest_training_dir / f'method_model_columns_{version}.json',
        'fighters_data_path': latest_training_dir / f'ufc_fighters_engineered_{version}.csv',
        'version': version
    }

def load_models_and_data():
    """Load the latest models and data."""
    print("📊 Loading latest models and data...")
    
    # Get latest model paths
    paths = get_latest_trained_models()
    
    # Load fighter data
    fighters_data_path = paths['fighters_data_path']
    if not fighters_data_path.exists():
        # Fallback to standard location
        fighters_data_path = Path('ufc_fighters_engineered_corrected.csv')
    
    fighters_df = pd.read_csv(fighters_data_path)
    print(f"✅ Loaded {len(fighters_df):,} fighter records")
    
    # Load column configurations
    with open(paths['winner_cols_path'], 'r') as f:
        winner_cols = json.load(f)
    
    with open(paths['method_cols_path'], 'r') as f:
        method_cols = json.load(f)
    
    print(f"✅ Loaded column configurations: {len(winner_cols)} winner features, {len(method_cols)} method features")
    
    # Load trained models
    winner_model = joblib.load(paths['winner_model_path'])
    method_model = joblib.load(paths['method_model_path'])
    
    print(f"✅ Loaded models from training version: {paths['version']}")
    print(f"   - Winner model: {winner_model.__class__.__name__}")
    print(f"   - Method model: {method_model.__class__.__name__}")
    
    return {
        'fighters_df': fighters_df,
        'winner_cols': winner_cols,
        'method_cols': method_cols,
        'winner_model': winner_model,
        'method_model': method_model,
        'version': paths['version'],
        'model_paths': paths
    }

# Load all models and data
model_data = load_models_and_data()
fighters_df = model_data['fighters_df']
winner_cols = model_data['winner_cols']
method_cols = model_data['method_cols']
winner_model = model_data['winner_model']
method_model = model_data['method_model']
print("\n✅ All models and data loaded successfully!")

In [ ]:
# Import prediction function with auto-reload
if 'src.prediction' in sys.modules:
    importlib.reload(sys.modules['src.prediction'])

try:
    from src.prediction import predict_fight_symmetrical
    print("✅ Successfully imported predict_fight_symmetrical!")
except ImportError as e:
    print(f"⚠️  Import error: {e}")
    print("💡 Please restart the Jupyter kernel and try again")
    
    # Define a fallback function if import fails
    def predict_fight_symmetrical(fighter1_name, fighter2_name, fighters_df, winner_cols, method_cols, winner_model, method_model):
        return {"error": "Function not available - please restart kernel and re-run cells"}

In [ ]:
def get_model_info():
    """Display comprehensive model information."""
    print("🤖 MODEL INFORMATION")
    print("=" * 50)
    
    print(f"Training Version: {model_data['version']}")
    print(f"Winner Model: {winner_model.__class__.__name__}")
    if hasattr(winner_model, 'n_estimators'):
        print(f"  - Trees: {winner_model.n_estimators}")
    if hasattr(winner_model, 'max_depth'):
        print(f"  - Max Depth: {winner_model.max_depth}")
    print(f"  - Features: {len(winner_cols)}")
    
    print(f"\nMethod Model: {method_model.__class__.__name__}")
    if hasattr(method_model, 'n_estimators'):
        print(f"  - Trees: {method_model.n_estimators}")
    if hasattr(method_model, 'max_depth'):
        print(f"  - Max Depth: {method_model.max_depth}")
    print(f"  - Features: {len(method_cols)}")
    print(f"  - Classes: {list(method_model.classes_)}")
    
    print(f"\nFighter Database: {len(fighters_df):,} fighters")
    print(f"Model Paths:")
    print(f"  - Winner: {Path(model_data['model_paths']['winner_model_path']).name}")
    print(f"  - Method: {Path(model_data['model_paths']['method_model_path']).name}")

# Display model information
get_model_info()

In [ ]:
def predict_single_fight(fighter1, fighter2, show_plot=True):
    """Predict a single fight with optional visualization."""
    print(f"🥊 Predicting: {fighter1} vs {fighter2}")
    print("="*50)
    
    result = predict_fight_symmetrical(
        fighter1, fighter2, fighters_df, 
        winner_cols, method_cols, winner_model, method_model
    )
    
    if 'error' in result:
        print(f"❌ Error: {result['error']}")
        
        # Show similar fighter names
        available_fighters = fighters_df['Name'].tolist()
        similar1 = [f for f in available_fighters if fighter1.lower() in f.lower()][:5]
        similar2 = [f for f in available_fighters if fighter2.lower() in f.lower()][:5]
        
        if similar1:
            print(f"Similar to '{fighter1}': {', '.join(similar1)}")
        if similar2:
            print(f"Similar to '{fighter2}': {', '.join(similar2)}")
        return None
    
    # Display results
    print(f"🏆 Predicted Winner: {result['predicted_winner']} ({result['winner_confidence']})")
    print(f"⚔️  Predicted Method: {result['predicted_method']}")
    
    print(f"\n📊 Win Probabilities:")
    for fighter, prob in result['win_probabilities'].items():
        print(f"   {fighter}: {prob}")
    
    print(f"\n🥊 Method Probabilities:")
    for method, prob in result['method_probabilities'].items():
        print(f"   {method}: {prob}")
    
    # Create visualization
    if show_plot:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # Winner probability pie chart
        fighters = list(result['win_probabilities'].keys())
        win_probs = [float(p.replace('%', '')) for p in result['win_probabilities'].values()]
        colors = ['#FF6B6B', '#4ECDC4']
        
        wedges, texts, autotexts = ax1.pie(win_probs, labels=fighters, autopct='%1.1f%%', 
                                          colors=colors, startangle=90, textprops={'fontsize': 12})
        ax1.set_title(f'Winner Prediction\n{result["predicted_winner"]} Wins', fontsize=14, fontweight='bold')
        
        # Method probability bar chart
        methods = list(result['method_probabilities'].keys())
        method_probs = [float(p.replace('%', '')) for p in result['method_probabilities'].values()]
        
        bars = ax2.bar(methods, method_probs, color=['#FFD93D', '#FF6B6B', '#6BCF7F'])
        ax2.set_title(f'Method Prediction\n{result["predicted_method"]}', fontsize=14, fontweight='bold')
        ax2.set_ylabel('Probability (%)', fontsize=12)
        ax2.set_ylim(0, 100)
        
        # Add value labels on bars
        for bar, prob in zip(bars, method_probs):
            ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                    f'{prob:.1f}%', ha='center', va='bottom', fontweight='bold')
        
        plt.tight_layout()
        plt.show()
    
    return result

print("📈 Single fight prediction function ready!")

In [ ]:
def predict_multiple_fights(fight_list, show_summary=True):
    """Predict multiple fights and show summary statistics."""
    results = {
        "event_summary": {
            "total_fights": len(fight_list),
            "processed_fights": 0,
            "failed_fights": 0,
            "prediction_timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        },
        "fight_predictions": [],
        "failed_predictions": [],
        "method_distribution": {"Decision": 0, "KO/TKO": 0, "Submission": 0}
    }
    
    print(f"🃏 Predicting {len(fight_list)} fights...")
    print("="*60)
    
    for i, fight_string in enumerate(fight_list, 1):
        if " vs " in fight_string:
            fighter_a, fighter_b = fight_string.split(" vs ")
            fighter_a, fighter_b = fighter_a.strip(), fighter_b.strip()
        elif " vs. " in fight_string:
            fighter_a, fighter_b = fight_string.split(" vs. ")
            fighter_a, fighter_b = fighter_a.strip(), fighter_b.strip()
        else:
            print(f"❌ Fight {i}: Invalid format '{fight_string}'")
            results["failed_predictions"].append({
                "fight": fight_string,
                "error": "Invalid format - use 'Fighter A vs Fighter B'"
            })
            results["event_summary"]["failed_fights"] += 1
            continue
        
        print(f"Fight {i}: {fighter_a} vs {fighter_b}")
        
        prediction = predict_fight_symmetrical(
            fighter_a, fighter_b, fighters_df,
            winner_cols, method_cols, winner_model, method_model
        )
        
        if 'error' in prediction:
            print(f"  ❌ {prediction['error']}")
            results["failed_predictions"].append({
                "fight": fight_string,
                "error": prediction['error']
            })
            results["event_summary"]["failed_fights"] += 1
        else:
            print(f"  🏆 Winner: {prediction['predicted_winner']} ({prediction['winner_confidence']})")
            print(f"  ⚔️  Method: {prediction['predicted_method']}")
            
            results["fight_predictions"].append(prediction)
            results["method_distribution"][prediction['predicted_method']] += 1
            results["event_summary"]["processed_fights"] += 1
        
        print()
    
    # Display summary
    if show_summary and results["fight_predictions"]:
        print("📊 EVENT SUMMARY")
        print("="*40)
        print(f"Total Fights: {results['event_summary']['total_fights']}")
        print(f"Successful Predictions: {results['event_summary']['processed_fights']}")
        print(f"Failed Predictions: {results['event_summary']['failed_fights']}")
        print(f"Success Rate: {(results['event_summary']['processed_fights']/results['event_summary']['total_fights'])*100:.1f}%")
        
        print(f"\n🥊 METHOD BREAKDOWN:")
        for method, count in results["method_distribution"].items():
            if count > 0:
                percentage = (count / results['event_summary']['processed_fights']) * 100
                print(f"  {method}: {count} fights ({percentage:.1f}%)")
        
        # Create method distribution chart
        methods = [k for k, v in results["method_distribution"].items() if v > 0]
        counts = [v for v in results["method_distribution"].values() if v > 0]
        
        if methods:
            plt.figure(figsize=(10, 6))
            colors = ['#FFD93D', '#FF6B6B', '#6BCF7F']
            bars = plt.bar(methods, counts, color=colors[:len(methods)])
            plt.title('Predicted Fight Methods Distribution', fontsize=14, fontweight='bold')
            plt.ylabel('Number of Fights', fontsize=12)
            
            # Add value labels
            for bar, count in zip(bars, counts):
                plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                        str(count), ha='center', va='bottom', fontweight='bold')
            
            plt.tight_layout()
            plt.show()
    
    return results

print("🃏 Multi-fight prediction function ready!")

In [ ]:
# Example: Single fight prediction
fighter1 = "Jon Jones"
fighter2 = "Stipe Miocic"

prediction = predict_single_fight(fighter1, fighter2, show_plot=True)

In [ ]:
# Define your UFC card fights
ufc_card_fights = [
    "Ilia Topuria vs. Charles Oliveira",
    "Alexandre Pantoja vs. Kai Kara-France",
    "Brandon Royval vs. Joshua Van",
    "Beneil Dariush vs. Renato Moicano",
    "Payton Talbott vs. Felipe Lima",
    "Niko Price vs. Jacobe Smith",
    "Jhonata Diniz vs. Alvin Hines",
    "Jack Hermansson vs. Gregory Rodrigues",
    "Hyder Amil vs. Jose Delgado",
    "Viviane Araujo vs. Tracy Cortez",
    "Terrance McKinney vs. Viacheslav Borshchev"
]

print("🎯 UFC Card Schedule:")
for i, fight in enumerate(ufc_card_fights, 1):
    print(f"   {i}. {fight}")

# Predict the entire card
card_results = predict_multiple_fights(ufc_card_fights, show_summary=True)

In [ ]:
# 🏆 UFC 304 FIGHT PREDICTIONS SETUP
# ===================================
# Generate predictions for the exact UFC 304 fight card

# UFC 304 Fight Card - August 3rd, 2024
ufc_304_fights = [
    "Amir Albazi vs Tatsuro Taira",          # Main Event (Flyweight)
    "Mateusz Rebecki vs Chris Duncan",        # Co-main (Lightweight)
    "Elves Brener vs Esteban Ribovics",      # Lightweight  
    "Karol Rosa vs Nora Cornolle",           # Women's Bantamweight
    "Neil Magny vs Elizeu Zaleski dos Santos", # Welterweight
    "Danny Silva vs Kevin Vallejos"          # Featherweight
]

print("🏆 UFC 304 - August 3rd, 2024")
print("🏟️  Manchester, England")
print("=" * 45)

for i, fight in enumerate(ufc_304_fights, 1):
    print(f"   {i}. {fight}")

print("\n🎯 Ready for predictions and profitability analysis!")

In [ ]:
# Profitability analysis setup
print("💰 Profitability analysis ready - using stealth TAB scraper!")

In [ ]:
# YOUR CUSTOM SINGLE FIGHT PREDICTION
# Modify these fighter names to predict any matchup

my_fighter1 = "Conor McGregor"
my_fighter2 = "Khabib Nurmagomedov"

my_prediction = predict_single_fight(my_fighter1, my_fighter2, show_plot=True)

In [ ]:
# YOUR CUSTOM CARD PREDICTIONS
# Add your own list of fights to predict

my_custom_card = [
    "Jon Jones vs. Stipe Miocic",
    "Islam Makhachev vs. Arman Tsarukyan",
    "Sean O'Malley vs. Merab Dvalishvili"
]

# Run predictions
my_card_results = predict_multiple_fights(my_custom_card, show_summary=True)